Что делать, когда программа работает не так как надо?
- print
- log
- debug
- иногда программа может работать с некоторыми некоректными данными, напрмер отсутствие колонки (таблицы) в базе данных...



Что делать, когда программа работает медленно?
- профилирование

Профилирование - форма динамического анализа программы. 

Основной способ сбора данных - замеры. 

Потом замеры обрабатываются для получиния какого-либо вывода.


Профилировать можно
- память
- время работы

Два подхода при профилировании

- сэмплирование (статистический профайлинг)

приостанавливаем выполнение программы каждые n милисекунд (секунд, наносекунд) записываем данные, обычно stacktrace и на основании этих данных получаем какую-то информацию

(+) более щедящий режим, каждые n милисекунд выполнение программы замедляется и затем продолжается в обычном режиме. Оставляет программе простор для работы обычным образом.

(-) не так подробно и не так точно


- инструментирование (cProfile/profile)

часть программы оборачивается определенным кодом. Например при вызове определенной функции пишет интересные нам данные

(+) более подробная информация о программе

(-) может занимать достаточно много ресурсов => вносит существенный след в процесс выполнения программы

In [ ]:
import logging

log = logging.getLogger(name)

# Если уже есть лог с таким же именем, то вернется он же
# Если такого обхекта еще нет, то он создастся и вернется

name = 'a.b.c'

# Обычно используют так:
log = logging.getLogger(__name__)

log.debug(...)
log.info(...)
log.warning(...)
log.error(...)
# log.exception(...)

try:
    ...
except ...:
    log.exception('Произошла странная ошибка.')

Конфигурирование логирования

гибкие настройки, например
- можно настроить логирование одновременно на нескольких языках
- можно дополнять логи дополнительной информацией, например веб сервер может писать id пользователя
- можно настроить запись логов в БД
- ...


- formatters
- filtres
- handlers
- loggers


Debugger (pdb)

```python3 -m pdb my_script.py```
 
```python
# start debugging here
import pdb
pdb.set_trace()
```

Команды:
```
- l (list)
- n (next)
- c (continue)
- s (step)
- r (return)
- b (break)
- cl (clear)
- any python code
```

pdb <-> ipython
```
ipython
%run -d my_script.py
```

pycharm

python3 -m cProfile my_script.py

cProfile.run('my_func()')

```python
import cProfile


def profile_this(f):
    def profiled_f(*args, **kwargs):
        prof = cProfile.Profile()
        result = prof.runcall(f, *args, **kwargs)
        f_path = f.__name__ + '.profile'
        prof.dump_stats(f_path)
        return result
    return profiled_f
```

```
>>> p = pstats.Stats('some.profile')
>>> p.sort_stats('time')
>>> p.print_stats(3)
```

cProfile + pstats + RunSnakeRun http://www.vrplumber.com/programming/runsnakerun/

Еще
- модуль timeit
- модуль traceback


Pickle (Бинарный формат)

Для распаковки вам могут подсунуть исполняемый код
https://docs.python.org/3/library/pickle.html#restricting-globals


Что можно сохранить/прочитать?
```
None, True, and False
integers, floating point numbers, complex numbers
strings, bytes, bytearrays
tuples, lists, sets, and dictionaries containing only picklable objects
functions defined at the top level of a module (using def, not lambda)
built-in functions defined at the top level of a module
classes that are defined at the top level of a module
instances of such classes whose __dict__ or the result of calling __getstate__() is picklable (see section Pickling Class Instances for details).
```
Хранятся только имена функций, классов...

Что происходит с объектами при сохранении/восстановлении?
```
By default, pickle will retrieve the class and the attributes of an instance via introspection. When a class instance is unpickled, its __init__() method is usually not invoked. The default behaviour first creates an uninitialized instance and then restores the saved attributes. The following code shows an implementation of this behaviour:

def save(obj):
    return (obj.__class__, obj.__dict__)

def load(cls, attributes):
    obj = cls.__new__(cls)
    obj.__dict__.update(attributes)
    return obj
```

Как использовать?
```
pickle.dump(obj, file, protocol=None)  # Pickler(file, protocol).dump(obj)
b = pickle.dumps(obj, protocol=None)
obj = pickle.load(file)                # Unpickler(file).load()
obj = pickle.loads(b)
```

В классе можно переопределить ```__setstate__```, ```__getstate__```
https://docs.python.org/3/library/pickle.html#handling-stateful-objects

Протоколы
```
There are currently 5 different protocols which can be used for pickling. The higher the protocol used, the more recent the version of Python needed to read the pickle produced.

Protocol version 0 is the original “human-readable” protocol and is backwards compatible with earlier versions of Python.
Protocol version 1 is an old binary format which is also compatible with earlier versions of Python.
Protocol version 2 was introduced in Python 2.3. It provides much more efficient pickling of new-style classes. Refer to PEP 307 for information about improvements brought by protocol 2.
Protocol version 3 was added in Python 3.0. It has explicit support for bytes objects and cannot be unpickled by Python 2.x. This is the default protocol, and the recommended protocol when compatibility with other Python 3 versions is required.
Protocol version 4 was added in Python 3.4. It adds support for very large objects, pickling more kinds of objects, and some data format optimizations. Refer to PEP 3154 for information about improvements brought by protocol 4.
```

Пример вычисления простых чисел с записью промежуточных результатов с испльзованием pickle
https://github.com/progpy/serialization/blob/v1/calculate_primes_and_save_pickle.py